<a href="https://colab.research.google.com/github/hugopoggi/pyspark_senai/blob/main/spark_24_n2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# 31/07/2024
# Criando ambiente Spark

# Instalando biblioteca pyspark
!pip install pyspark

In [46]:
# Importando funções e criando o objeto spark:
from pyspark.sql import SparkSession as ss
from pyspark.sql import functions as f
# Importando função para converter dados
from pyspark.sql.types import DoubleType

In [47]:
spark = ss.builder.appName("SENAI").getOrCreate()

In [48]:
spark

In [49]:
# Criando um data frame manualmente
dados = [
    ("joao", "analista", 5000),
    ("isabela", "analista", 5280),
    ("julia", "assistente", 2800),
    ("kleber", "auxiliar", 2200)
]

In [50]:
colunas = ["func", "cargo", "salario"]

In [51]:
# Criando Data Frame (estrutura de linhas x colunas):
df = spark.createDataFrame(dados, colunas)

In [52]:
# Explorando Data Frame:
# Verificando nome das colunas e seus tipos:
df.printSchema()

root
 |-- func: string (nullable = true)
 |-- cargo: string (nullable = true)
 |-- salario: long (nullable = true)



In [53]:
# Exibindo dados do data frame.
df.show()

+-------+----------+-------+
|   func|     cargo|salario|
+-------+----------+-------+
|   joao|  analista|   5000|
|isabela|  analista|   5280|
|  julia|assistente|   2800|
| kleber|  auxiliar|   2200|
+-------+----------+-------+



In [54]:
df.toPandas()

,func,cargo,salario
0,joao,analista,5000
1,isabela,analista,5280
2,julia,assistente,2800
3,kleber,auxiliar,2200


In [55]:
# Unindo dois DataFrames
dados2 = [
    ("ana", "auxiliar", 2490),
    ("rogerio", "auxiliar", 9800),
    ("roberto", "diretor", 32000)
]

In [56]:
df2 = spark.createDataFrame(dados2, colunas)

In [57]:
# Unindo data frames:
df_total = df.union(df2)

In [58]:
df_total.show()

+-------+----------+-------+
|   func|     cargo|salario|
+-------+----------+-------+
|   joao|  analista|   5000|
|isabela|  analista|   5280|
|  julia|assistente|   2800|
| kleber|  auxiliar|   2200|
|    ana|  auxiliar|   2490|
|rogerio|  auxiliar|   9800|
|roberto|   diretor|  32000|
+-------+----------+-------+



In [59]:
df = df_total

In [60]:
df.show()

+-------+----------+-------+
|   func|     cargo|salario|
+-------+----------+-------+
|   joao|  analista|   5000|
|isabela|  analista|   5280|
|  julia|assistente|   2800|
| kleber|  auxiliar|   2200|
|    ana|  auxiliar|   2490|
|rogerio|  auxiliar|   9800|
|roberto|   diretor|  32000|
+-------+----------+-------+



In [61]:
df.toPandas()

,func,cargo,salario
0,joao,analista,5000
1,isabela,analista,5280
2,julia,assistente,2800
3,kleber,auxiliar,2200
4,ana,auxiliar,2490
5,rogerio,auxiliar,9800
6,roberto,diretor,32000


In [62]:
# Filtrando valores:
# Motrar somente 'analistas' do DataFrame:
df.where('cargo =="analista"').show()

+-------+--------+-------+
|   func|   cargo|salario|
+-------+--------+-------+
|   joao|analista|   5000|
|isabela|analista|   5280|
+-------+--------+-------+



In [63]:
 # Filtrando valores:
# Motrar somente 'analistas' que ganham mais de 5000 do DataFrame:
df.where('cargo =="analista" and salario > 5000').show()

+-------+--------+-------+
|   func|   cargo|salario|
+-------+--------+-------+
|isabela|analista|   5280|
+-------+--------+-------+



In [64]:
df.show()

+-------+----------+-------+
|   func|     cargo|salario|
+-------+----------+-------+
|   joao|  analista|   5000|
|isabela|  analista|   5280|
|  julia|assistente|   2800|
| kleber|  auxiliar|   2200|
|    ana|  auxiliar|   2490|
|rogerio|  auxiliar|   9800|
|roberto|   diretor|  32000|
+-------+----------+-------+



In [65]:
# Criando ou alterando(mesmo nome altera valor da msm colula nome
# diferente cria coluna nova) novas colunas no pyspark:
# Visualizando alteração:
df.withColumn("novo_salario", df.salario * 1.07).show()

+-------+----------+-------+------------+
|   func|     cargo|salario|novo_salario|
+-------+----------+-------+------------+
|   joao|  analista|   5000|      5350.0|
|isabela|  analista|   5280|      5649.6|
|  julia|assistente|   2800|      2996.0|
| kleber|  auxiliar|   2200|      2354.0|
|    ana|  auxiliar|   2490|      2664.3|
|rogerio|  auxiliar|   9800|     10486.0|
|roberto|   diretor|  32000|     34240.0|
+-------+----------+-------+------------+



In [66]:
# Efetivando alteração no data frame
df = df.withColumn("novo_salario", df.salario * 1.07)

In [67]:
df.show()

+-------+----------+-------+------------+
|   func|     cargo|salario|novo_salario|
+-------+----------+-------+------------+
|   joao|  analista|   5000|      5350.0|
|isabela|  analista|   5280|      5649.6|
|  julia|assistente|   2800|      2996.0|
| kleber|  auxiliar|   2200|      2354.0|
|    ana|  auxiliar|   2490|      2664.3|
|rogerio|  auxiliar|   9800|     10486.0|
|roberto|   diretor|  32000|     34240.0|
+-------+----------+-------+------------+



In [68]:
# Criando DataFrame via importação de dados / arquivos:
url = "/content/drive/MyDrive/BasesSpark/empresas/part*"

empresas = spark.read.csv(
    url,
    sep= ";",
    header= True,
    # InferSchema "tenta" entender o tipo de dado e trazer em seu formato
    inferSchema= True
)

In [69]:
# Numero de linhas e trunca os dados falsos
empresas.show(20, False)

+-----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|cnpj |razao_social                                                                                |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|306  |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                      |0,00          |1    |NULL           |
|1355 |BRASILEIRO & OLIVEIRA LTDA                                                                  |2062             |49                      |0,00          |5    |NULL           |
|4820 |REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E TABELIONATO E REGISTRO DE CONSTRATOS MARIT

In [70]:
# Verificando quantos registros tem no dataframe:
empresas.count()

4585679

In [71]:
empresas.printSchema()

root
 |-- cnpj: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_responsavel: integer (nullable = true)
 |-- capital_social: string (nullable = true)
 |-- porte: integer (nullable = true)
 |-- ente_federativo: string (nullable = true)



In [72]:
# Convertendo uma coluna de string para decimal:
# Antes de converter precisamos substituir a virgula pelo ponto.

empresas.withColumn("capital_social", f.regexp_replace("capital_social", ",", ".")).show()

+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
| cnpj|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|                      49|          0.00|    1|           NULL|
| 1355|BRASILEIRO & OLIV...|             2062|                      49|          0.00|    5|           NULL|
| 4820|REGISTRO DE IMOVE...|             3034|                      32|          0.00|    5|           NULL|
| 5347|ROSELY APARECIDA ...|             2135|                      50|          0.00|    5|           NULL|
| 6846|BADU E FILHOS TEC...|             2062|                      49|       4000.00|    1|           NULL|
| 8416|  ELETRICA RUBI LTDA|             2062|                      49|          0.00|    5|           NULL|
| 8992|SHIROMA VEIC

In [73]:
empresas = empresas.withColumn("capital_social", f.regexp_replace("capital_social", ",", "."))

In [74]:
empresas.show()

+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
| cnpj|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|                      49|          0.00|    1|           NULL|
| 1355|BRASILEIRO & OLIV...|             2062|                      49|          0.00|    5|           NULL|
| 4820|REGISTRO DE IMOVE...|             3034|                      32|          0.00|    5|           NULL|
| 5347|ROSELY APARECIDA ...|             2135|                      50|          0.00|    5|           NULL|
| 6846|BADU E FILHOS TEC...|             2062|                      49|       4000.00|    1|           NULL|
| 8416|  ELETRICA RUBI LTDA|             2062|                      49|          0.00|    5|           NULL|
| 8992|SHIROMA VEIC

In [75]:
# Convertendo a coluna "capital_social" para float (DoubleType):
empresas.withColumn("capital_social",
                    empresas.capital_social.cast(DoubleType())
                    ).show()

+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
| cnpj|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|                      49|           0.0|    1|           NULL|
| 1355|BRASILEIRO & OLIV...|             2062|                      49|           0.0|    5|           NULL|
| 4820|REGISTRO DE IMOVE...|             3034|                      32|           0.0|    5|           NULL|
| 5347|ROSELY APARECIDA ...|             2135|                      50|           0.0|    5|           NULL|
| 6846|BADU E FILHOS TEC...|             2062|                      49|        4000.0|    1|           NULL|
| 8416|  ELETRICA RUBI LTDA|             2062|                      49|           0.0|    5|           NULL|
| 8992|SHIROMA VEIC

In [76]:
empresas = empresas.withColumn("capital_social",
                    empresas.capital_social.cast(DoubleType())
                    )

In [77]:
empresas.show()

+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
| cnpj|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|                      49|           0.0|    1|           NULL|
| 1355|BRASILEIRO & OLIV...|             2062|                      49|           0.0|    5|           NULL|
| 4820|REGISTRO DE IMOVE...|             3034|                      32|           0.0|    5|           NULL|
| 5347|ROSELY APARECIDA ...|             2135|                      50|           0.0|    5|           NULL|
| 6846|BADU E FILHOS TEC...|             2062|                      49|        4000.0|    1|           NULL|
| 8416|  ELETRICA RUBI LTDA|             2062|                      49|           0.0|    5|           NULL|
| 8992|SHIROMA VEIC

In [78]:
empresas.printSchema()

root
 |-- cnpj: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_responsavel: integer (nullable = true)
 |-- capital_social: double (nullable = true)
 |-- porte: integer (nullable = true)
 |-- ente_federativo: string (nullable = true)



In [79]:
empresas.limit(30).toPandas()

,cnpj,razao_social,natureza_juridica,qualificacao_responsavel,capital_social,porte,ente_federativo
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.0,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.0,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.0,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.0,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.0,1,None
5,8416,ELETRICA RUBI LTDA,2062,49,0.0,5,None
6,8992,SHIROMA VEICULOS LTDA.,2062,49,0.0,5,None
7,9091,CONTATOS BAR E LANCHONETE LTDA,2062,49,0.0,5,None
8,9614,ANTONIA APARECIDA DE SOUZA ULIANA,2135,50,0.0,5,None
9,9896,DORACY CORAT DA COSTA,2135,50,0.0,5,None


In [80]:
# Entendendo coluna "ente_federativo":
empresas\
        .where("ente_federativo is not null")\
        .show(20, False)

+--------+----------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------------------+
|cnpj    |razao_social                                                                                  |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo            |
+--------+----------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------------------+
|784274  |SECRETARIA MUNICIPAL DE SAUDE                                                                 |1031             |5                       |0.0           |5    |PARATINGA - BA             |
|2030715 |AGENCIA NACIONAL DE TELECOMUNICACOES                                                          |1104             |16                      |0.0           |5    |UNI�O                      |
|3104818 |

In [81]:
# Padronizando dados nulos da coluna "ente_federativo"
empresas.na.fill({"ente_federativo":"N/A"}).show()

+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
| cnpj|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|                      49|           0.0|    1|            N/A|
| 1355|BRASILEIRO & OLIV...|             2062|                      49|           0.0|    5|            N/A|
| 4820|REGISTRO DE IMOVE...|             3034|                      32|           0.0|    5|            N/A|
| 5347|ROSELY APARECIDA ...|             2135|                      50|           0.0|    5|            N/A|
| 6846|BADU E FILHOS TEC...|             2062|                      49|        4000.0|    1|            N/A|
| 8416|  ELETRICA RUBI LTDA|             2062|                      49|           0.0|    5|            N/A|
| 8992|SHIROMA VEIC

In [82]:
empresas = empresas.na.fill({"ente_federativo":"N/A"})

In [83]:
# Verifique quantas empresas tem capital_social igual ou menor a zero:

empresas\
        .where("capital_social < 0").count()

0

In [84]:
empresas\
        .where("capital_social == 0")\
        .count()

1691023

In [85]:
# Criando um novo Data Frame somente com capital social maior que zero:
capitalizadas = empresas\
        .where("capital_social > 0")


In [86]:
capitalizadas.show(30, False)

+------+-------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|cnpj  |razao_social                                                       |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+------+-------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|6846  |BADU E FILHOS TECIDOS LTDA                                         |2062             |49                      |4000.0        |1    |N/A            |
|24205 |SUELY LEME MARI ANI 25086572800                                    |2135             |50                      |1000.0        |1    |N/A            |
|58970 |TOTAL CAR VEICULOS LTDA                                            |2062             |49                      |20000.0       |1    |N/A            |
|70273 |VANDA DA SILVA LANCHONETE                         

In [87]:
# Pesquisando strings:
# Mostre em tela os 20 primeiros registros de empresas que contenham a palavra
# restaurante
capitalizadas\
              .where("razao_social like '%RESTAURANTE%'")\
              .show(20, False)

+--------+------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|cnpj    |razao_social                                          |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+--------+------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|2591875 |BAR E RESTAURANTE CASA DA QUINTA LTDA                 |2062             |49                      |5000.0        |1    |N/A            |
|3059084 |DON MUGO RESTAURANTE LTDA                             |2062             |49                      |10000.0       |1    |N/A            |
|3396661 |GERACAO DE OURO - BAR E RESTAURANTE LTDA              |2062             |5                       |40000.0       |5    |N/A            |
|5930238 |PSTJ RESTAURANTE LTDA                                 |2062             |49                      |30000.0       |1

In [88]:
# Trabalhando com dados estatisticos basico:
# Crie uma visão estatistica geral contendo: contagem, média. mediana, soma e
# Desvio padrão da coluna capital_social.

capitalizadas.agg(
          f.count("*").alias("quantidade"),
          f.mean("capital_social").alias("media_investimento"),
          f.median("capital_social").alias("mediana_investimento"),
          f.sum("capital_social").alias("total_investido"),
          f.stddev("capital_social").alias("desvio_padrao")
          ).show()



+----------+------------------+--------------------+--------------------+-------------------+
|quantidade|media_investimento|mediana_investimento|     total_investido|      desvio_padrao|
+----------+------------------+--------------------+--------------------+-------------------+
|   2894656| 797946.8062905608|              5000.0|2.309781510509809...|2.658089851944688E8|
+----------+------------------+--------------------+--------------------+-------------------+



In [89]:
capitalizadas.agg(
          f.format_number(f.count("*"),2).alias("quantidade"),
          f.format_number(f.mean("capital_social"),2).alias("media_investimento"),
          f.format_number(f.median("capital_social"),2).alias("mediana_investimento"),
          f.format_number(f.sum("capital_social"),2).alias("total_investido"),
          f.format_number(f.stddev("capital_social"),2).alias("desvio_padrao")
          ).toPandas()

,quantidade,media_investimento,mediana_investimento,total_investido,desvio_padrao
0,"2,894,656.00","797,946.81","5,000.00","2,309,781,510,509.81","265,808,985.19"


In [90]:
# Mostre as mesmas informações acima mas agrupando por porte de empresa
empresas.show()

+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
| cnpj|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|                      49|           0.0|    1|            N/A|
| 1355|BRASILEIRO & OLIV...|             2062|                      49|           0.0|    5|            N/A|
| 4820|REGISTRO DE IMOVE...|             3034|                      32|           0.0|    5|            N/A|
| 5347|ROSELY APARECIDA ...|             2135|                      50|           0.0|    5|            N/A|
| 6846|BADU E FILHOS TEC...|             2062|                      49|        4000.0|    1|            N/A|
| 8416|  ELETRICA RUBI LTDA|             2062|                      49|           0.0|    5|            N/A|
| 8992|SHIROMA VEIC

In [91]:
capitalizadas.groupby("porte").agg(
          f.format_number(f.count("*"),2).alias("quantidade"),
          f.format_number(f.mean("capital_social"),2).alias("media_investimento"),
          f.format_number(f.median("capital_social"),2).alias("mediana_investimento"),
          f.format_number(f.sum("capital_social"),2).alias("total_investido"),
          f.format_number(f.stddev("capital_social"),2).alias("desvio_padrao")
          ).toPandas()

,porte,quantidade,media_investimento,mediana_investimento,total_investido,desvio_padrao
0,NaN,1.00,"50,000.00","50,000.00","50,000.00",None
1,1.0,"2,671,059.00","398,290.53","5,000.00","1,063,857,513,944.56","222,606,539.80"
2,3.0,"102,894.00","2,910,839.84","67,800.00","299,507,954,553.49","790,882,852.95"
3,5.0,"120,702.00","7,840,930.49","50,000.00","946,415,992,011.76","254,102,444.49"


In [92]:
# Altere a coluna "porte" para seu descritivo
# Consultar dicionário de dados:
capitalizadas.withColumn("porte",
                         f.when(capitalizadas.porte == 0, "NAO INFORMADO")\
                         .when(capitalizadas.porte == 1, "MICRO EMPRESA")\
                         .when(capitalizadas.porte == 3, "PEQUENO PORTE")\
                         .otherwise("OUTRAS")
                         ).show()

+------+--------------------+-----------------+------------------------+--------------+-------------+---------------+
|  cnpj|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|        porte|ente_federativo|
+------+--------------------+-----------------+------------------------+--------------+-------------+---------------+
|  6846|BADU E FILHOS TEC...|             2062|                      49|        4000.0|MICRO EMPRESA|            N/A|
| 24205|SUELY LEME MARI A...|             2135|                      50|        1000.0|MICRO EMPRESA|            N/A|
| 58970|TOTAL CAR VEICULO...|             2062|                      49|       20000.0|MICRO EMPRESA|            N/A|
| 70273|VANDA DA SILVA LA...|             2135|                      50|       15000.0|MICRO EMPRESA|            N/A|
| 74218|   V G SILVA TECIDOS|             2135|                      50|       16000.0|MICRO EMPRESA|            N/A|
|116791| AGNALDO ROCHA VIANA|             2135|         

In [93]:
capitalizadas = capitalizadas.withColumn("porte",
                         f.when(capitalizadas.porte == 0, "NAO INFORMADO")\
                         .when(capitalizadas.porte == 1, "MICRO EMPRESA")\
                         .when(capitalizadas.porte == 3, "PEQUENO PORTE")\
                         .otherwise("OUTRAS")
                         )

In [94]:
capitalizadas.groupby("porte").agg(
          f.format_number(f.count("*"),2).alias("quantidade"),
          f.format_number(f.mean("capital_social"),2).alias("media_investimento"),
          f.format_number(f.median("capital_social"),2).alias("mediana_investimento"),
          f.format_number(f.sum("capital_social"),2).alias("total_investido"),
          f.format_number(f.stddev("capital_social"),2).alias("desvio_padrao")
          ).orderBy("porte").toPandas()

,porte,quantidade,media_investimento,mediana_investimento,total_investido,desvio_padrao
0,MICRO EMPRESA,"2,671,059.00","398,290.53","5,000.00","1,063,857,513,944.56","222,606,539.80"
1,OUTRAS,"120,703.00","7,840,865.94","50,000.00","946,416,042,011.76","254,101,392.88"
2,PEQUENO PORTE,"102,894.00","2,910,839.84","67,800.00","299,507,954,553.49","790,882,852.95"


In [95]:
# Altere a coluna "qualificação_responsavel" para "responsavel" e a coluna:
# "ente_federativo" para "UF":
capitalizadas.withColumnsRenamed({
    "qualificacao_responsavel":"responsavel",
    "ente_federativo":"UF"
}).show()



+------+--------------------+-----------------+-----------+--------------+-------------+---+
|  cnpj|        razao_social|natureza_juridica|responsavel|capital_social|        porte| UF|
+------+--------------------+-----------------+-----------+--------------+-------------+---+
|  6846|BADU E FILHOS TEC...|             2062|         49|        4000.0|MICRO EMPRESA|N/A|
| 24205|SUELY LEME MARI A...|             2135|         50|        1000.0|MICRO EMPRESA|N/A|
| 58970|TOTAL CAR VEICULO...|             2062|         49|       20000.0|MICRO EMPRESA|N/A|
| 70273|VANDA DA SILVA LA...|             2135|         50|       15000.0|MICRO EMPRESA|N/A|
| 74218|   V G SILVA TECIDOS|             2135|         50|       16000.0|MICRO EMPRESA|N/A|
|116791| AGNALDO ROCHA VIANA|             2135|         50|       60000.0|MICRO EMPRESA|N/A|
|122983|      ALOISIO OCZUST|             2135|         50|       10000.0|MICRO EMPRESA|N/A|
|164903|SUPERMERCADO DALT...|             2062|         49|       1000

In [96]:
capitalizadas = capitalizadas.withColumnsRenamed({
    "qualificacao_responsavel":"responsavel",
    "ente_federativo":"UF"
})

In [97]:
capitalizadas.show()

+------+--------------------+-----------------+-----------+--------------+-------------+---+
|  cnpj|        razao_social|natureza_juridica|responsavel|capital_social|        porte| UF|
+------+--------------------+-----------------+-----------+--------------+-------------+---+
|  6846|BADU E FILHOS TEC...|             2062|         49|        4000.0|MICRO EMPRESA|N/A|
| 24205|SUELY LEME MARI A...|             2135|         50|        1000.0|MICRO EMPRESA|N/A|
| 58970|TOTAL CAR VEICULO...|             2062|         49|       20000.0|MICRO EMPRESA|N/A|
| 70273|VANDA DA SILVA LA...|             2135|         50|       15000.0|MICRO EMPRESA|N/A|
| 74218|   V G SILVA TECIDOS|             2135|         50|       16000.0|MICRO EMPRESA|N/A|
|116791| AGNALDO ROCHA VIANA|             2135|         50|       60000.0|MICRO EMPRESA|N/A|
|122983|      ALOISIO OCZUST|             2135|         50|       10000.0|MICRO EMPRESA|N/A|
|164903|SUPERMERCADO DALT...|             2062|         49|       1000

In [101]:
# Alterando e visualizando colunas
capitalizadas.withColumnsRenamed({
    "razao_social":"nome_empresa",
  "capital_social":"valor_investido"
}).show()

+------+--------------------+-----------------+-----------+---------------+-------------+---+
|  cnpj|        nome_empresa|natureza_juridica|responsavel|valor_investido|        porte| UF|
+------+--------------------+-----------------+-----------+---------------+-------------+---+
|  6846|BADU E FILHOS TEC...|             2062|         49|         4000.0|MICRO EMPRESA|N/A|
| 24205|SUELY LEME MARI A...|             2135|         50|         1000.0|MICRO EMPRESA|N/A|
| 58970|TOTAL CAR VEICULO...|             2062|         49|        20000.0|MICRO EMPRESA|N/A|
| 70273|VANDA DA SILVA LA...|             2135|         50|        15000.0|MICRO EMPRESA|N/A|
| 74218|   V G SILVA TECIDOS|             2135|         50|        16000.0|MICRO EMPRESA|N/A|
|116791| AGNALDO ROCHA VIANA|             2135|         50|        60000.0|MICRO EMPRESA|N/A|
|122983|      ALOISIO OCZUST|             2135|         50|        10000.0|MICRO EMPRESA|N/A|
|164903|SUPERMERCADO DALT...|             2062|         49| 

In [102]:
# Aplicando alteração em determinadas colunas
capitalizadas = capitalizadas.withColumnsRenamed({
    "razao_social":"nome_empresa",
    "capital_social":"valor_investido"
})

In [104]:
capitalizadas.show(50,False)

+-------+-------------------------------------------------------------------+-----------------+-----------+---------------+-------------+---+
|cnpj   |nome_empresa                                                       |natureza_juridica|responsavel|valor_investido|porte        |UF |
+-------+-------------------------------------------------------------------+-----------------+-----------+---------------+-------------+---+
|6846   |BADU E FILHOS TECIDOS LTDA                                         |2062             |49         |4000.0         |MICRO EMPRESA|N/A|
|24205  |SUELY LEME MARI ANI 25086572800                                    |2135             |50         |1000.0         |MICRO EMPRESA|N/A|
|58970  |TOTAL CAR VEICULOS LTDA                                            |2062             |49         |20000.0        |MICRO EMPRESA|N/A|
|70273  |VANDA DA SILVA LANCHONETE                                          |2135             |50         |15000.0        |MICRO EMPRESA|N/A|
|74218

In [115]:
# Contabilizando empresas em Recuperação judicial
capitalizadas.where("nome_empresa like 'RJ %'").count()

172

In [118]:
#Filtrando empresas com valor investido entre R$500.000,00 e
# R$1.000.000,00
capitalizadas.where("valor_investido > 500000 AND valor_investido <= 1000000")\
.show()

+-------+--------------------+-----------------+-----------+---------------+-------------+---+
|   cnpj|        nome_empresa|natureza_juridica|responsavel|valor_investido|        porte| UF|
+-------+--------------------+-----------------+-----------+---------------+-------------+---+
| 346063|COR DO CEU INDUST...|             2062|         49|      1000000.0|PEQUENO PORTE|N/A|
| 992392|   RUDIMAR BORGHETTI|             2135|         50|       600000.0|MICRO EMPRESA|N/A|
|1355433|PREV INVEST CONST...|             2305|         65|      1000000.0|MICRO EMPRESA|N/A|
|2315381|EXATAS INCORPORAD...|             2062|         49|       800000.0|PEQUENO PORTE|N/A|
|3328068|ARLI ANTONIO BERG...|             2062|         49|       530000.0|MICRO EMPRESA|N/A|
|3383910|SUCUARANA FLOREST...|             2062|         49|       650000.0|PEQUENO PORTE|N/A|
|4300654|R R CONSTRUCOES E...|             2062|         49|       600000.0|PEQUENO PORTE|N/A|
|4877029| UBX ENTERPRISE LTDA|             2062|  

In [119]:
# Criando dataframe de empresas com valor investido entre R$500.000,00 e
# R$1.000.000,00
alto_valor = capitalizadas.where("valor_investido > 500000 and valor_investido <= 1000000")

In [120]:
alto_valor.show()

+-------+--------------------+-----------------+-----------+---------------+-------------+---+
|   cnpj|        nome_empresa|natureza_juridica|responsavel|valor_investido|        porte| UF|
+-------+--------------------+-----------------+-----------+---------------+-------------+---+
| 346063|COR DO CEU INDUST...|             2062|         49|      1000000.0|PEQUENO PORTE|N/A|
| 992392|   RUDIMAR BORGHETTI|             2135|         50|       600000.0|MICRO EMPRESA|N/A|
|1355433|PREV INVEST CONST...|             2305|         65|      1000000.0|MICRO EMPRESA|N/A|
|2315381|EXATAS INCORPORAD...|             2062|         49|       800000.0|PEQUENO PORTE|N/A|
|3328068|ARLI ANTONIO BERG...|             2062|         49|       530000.0|MICRO EMPRESA|N/A|
|3383910|SUCUARANA FLOREST...|             2062|         49|       650000.0|PEQUENO PORTE|N/A|
|4300654|R R CONSTRUCOES E...|             2062|         49|       600000.0|PEQUENO PORTE|N/A|
|4877029| UBX ENTERPRISE LTDA|             2062|  

In [123]:
# Criando filtro empresas de pequeno porte com investimento maior que R$500.000,00
capitalizadas.where("valor_investido > 500000 and porte like 'PEQUENO PORTE'").show()

+--------+--------------------+-----------------+-----------+---------------+-------------+---+
|    cnpj|        nome_empresa|natureza_juridica|responsavel|valor_investido|        porte| UF|
+--------+--------------------+-----------------+-----------+---------------+-------------+---+
|  346063|COR DO CEU INDUST...|             2062|         49|      1000000.0|PEQUENO PORTE|N/A|
| 1766239|M COELHO DOS SANT...|             2062|         49|      6000000.0|PEQUENO PORTE|N/A|
| 2315381|EXATAS INCORPORAD...|             2062|         49|       800000.0|PEQUENO PORTE|N/A|
| 3383910|SUCUARANA FLOREST...|             2062|         49|       650000.0|PEQUENO PORTE|N/A|
| 4169221|C S I CONSTRUTORA...|             2062|         49|      6200000.0|PEQUENO PORTE|N/A|
| 4196988|PORTAL EMPREENDIM...|             2062|         49|      1548000.0|PEQUENO PORTE|N/A|
| 4300654|R R CONSTRUCOES E...|             2062|         49|       600000.0|PEQUENO PORTE|N/A|
| 4404297|AGROPART - IMOBIL...|         

In [124]:
# Criando dataframe de empresas de pequeno porte com investimento superior à
# R$500.000,00
pp_alto_valor = capitalizadas.where("valor_investido > 500000 and porte like 'PEQUENO PORTE'")

In [127]:
# Visualizando novo Data Frame ordenando por valor investido:
pp_alto_valor.orderBy("valor_investido").toPandas()

,cnpj,nome_empresa,natureza_juridica,responsavel,valor_investido,porte,UF
0,4853219,PREMIUM OPERADOR LOGISTICO LTDA,2062,49,5.000020e+05,PEQUENO PORTE,N/A
1,33704783,PETROCENTER REPRESENTACAO COMERCIAL DE RESIDU...,2062,49,5.001000e+05,PEQUENO PORTE,N/A
2,83298216,POSMOVIL-POSTO DE MOLAS VIVIAN LTDA,2062,49,5.010000e+05,PEQUENO PORTE,N/A
3,86708609,DIGEM DIAGNOSTICO POR IMAGEM LTDA,2062,49,5.010000e+05,PEQUENO PORTE,N/A
4,886621,HOTEL SAO BRAZ EIRELI,2305,65,5.010000e+05,PEQUENO PORTE,N/A
...,...,...,...,...,...,...,...
5216,20355297,DIAS DEFENSE SEGURANCA PATRIMONIAL LTDA.,2062,49,1.000000e+08,PEQUENO PORTE,N/A
5217,15250211,MINERACAO INTERNACIONAL LTDA,2062,49,1.000000e+08,PEQUENO PORTE,N/A
5218,31107871,ERVA DOCE AGRONEGOCIOS LTDA,2062,49,1.300000e+08,PEQUENO PORTE,N/A
5219,6352117,REBOUCAS E CIA LTDA.,2062,49,2.920270e+10,PEQUENO PORTE,N/A
